# Classification of Human vs LLM Generated Text Using BoW Perceptron

The following notebook implements a single perceptron BoW classifier to determine if text is generated by a human or a LLM.

First lets set up our environment. Import libraries, initalize recorders and seeds, etc.

In [1]:
# Standard ML libaries
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report

# Custom utils for recording model progress.
from utils.recorder_util import ModelResults

In [2]:
# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

model_name = "single_perceptron_bow"
author = "ben_secekeler"

results = ModelResults(model_name, author, seed)

device: cuda
random seed: 1234


Next, lets load in the complete dataset using Pandas. We will split it into a 80-20 training-testing set 

In [3]:
train_data = pd.read_csv("../trainData/trainData.csv")
test_data = pd.read_csv("../testData/testData.csv")

train_data.head()

,label,source,text
0,0,1,Help wanted!\n\nThe Seagoing Cowboys program i...
1,0,1,The system of the Electoral College is a widel...
2,1,1,The renowned British statesman Winston Churchi...
3,0,1,"My grandfather would always say ""creativity is..."
4,1,1,In my pursuit to become an assistant manager a...


Preprocess data.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(train_data["text"])

x_train = cv.transform(train_data["text"])
y_train = train_data["label"]

x_test = cv.transform(test_data["text"])
y_test = test_data["label"]

print("training: ")
print("x : " + str(x_train.get_shape()))
print("y : " + str(y_train.size))

print("testing: ")
print("x : " + str(x_test.get_shape()))
print("y : " + str(y_test.size))

training: 
x : (36997, 68854)
y : 36997
testing: 
x : (6870, 68854)
y : 6870


In [5]:
from torch.utils.data import Dataset
from scipy import sparse
class BotFinderDataset (Dataset):
	def __init__(self, x, y):
		self.x = x
		self.y = y

		print(type(x))
		s = x.tocoo()
		i = torch.LongTensor(np.vstack((s.row, s.col)))
		v = torch.FloatTensor(s.data)

		self.x_tensor = torch.sparse.FloatTensor(i, v, torch.Size(s.shape))

	def __len__(self):
		return len(self.y)
	
	def __getitem__(self, index):
		# x = self.x[index]

		x = self.x_tensor[index].to_dense()
		y = self.y[index]

		return (x, y)
	
data_test = BotFinderDataset(x_train, y_train)
data_test[3][0]


<class 'scipy.sparse._csr.csr_matrix'>


/tmp/ipykernel_24473/3913667437.py:13: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /opt/conda/conda-bld/pytorch_1729647329220/work/torch/csrc/utils/tensor_new.cpp:651.)
  self.x_tensor = torch.sparse.FloatTensor(i, v, torch.Size(s.shape))


tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [6]:
from torch import nn
import torch.nn.functional as F

class PerceptronModel(nn.Module):
	def __init__(self, input_dim):
		super().__init__()
		self.layers = nn.Sequential(nn.Linear(input_dim, 2))
	
	def forward(self, x):
		return self.layers(x)

In [7]:
from torch import optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

model = PerceptronModel(x_train.shape[1]).to(device=device)



loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam (
	model.parameters(),
	lr = 0.001,
	weight_decay = 0.0001
)

train_ds = BotFinderDataset(
	x_train,
	y_train
)
test_ds = BotFinderDataset(
	x_test,
	y_test
)

train_dl = DataLoader(train_ds, 
					  batch_size = 10, 
					  shuffle = True, 
					  )
test_dl = DataLoader(test_ds, 
					  batch_size = 10, 
					  shuffle = True)

print(len(train_dl))
print(len(test_dl))

<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
3700
687


In [8]:
# training
for epoch in range(2):
	print(f"epoch {epoch}")

	model.train()

	# for X, y_true in tqdm(train_dl):
	for X, y_true in train_dl:
		model.zero_grad()
		X = X.to(device)
		y_true = y_true.to(device)
		
		y_pred = model(X)

		loss = loss_function(y_pred, y_true)

		loss.backward()

		optimizer.step()

epoch 0
epoch 1


In [9]:
data_loader = test_dl
y_pred = []

model.eval()

# disable gradient calculation
with torch.no_grad():
    for X, _ in (data_loader):
        X = X.to(device)
        # predict one class per example
        y = torch.argmax(model(X), dim=1)
        # convert tensor to numpy array
        y_pred.append(y.cpu().numpy())
    
# print results
y_true = y_test
y_pred = np.concatenate(y_pred)
print(classification_report(y_true, y_pred, target_names=['human', 'bot'], digits = 3))

              precision    recall  f1-score   support

       human      0.502     0.501     0.502      3435
         bot      0.502     0.504     0.503      3435

    accuracy                          0.502      6870
   macro avg      0.502     0.502     0.502      6870
weighted avg      0.502     0.502     0.502      6870

